In [25]:
import numpy as np
import pandas as pd

result = pd.read_csv('sample/preprocessed_dataset.csv')
print(result)
data = result.drop('파일명',axis=1)
data = data.drop('선종', axis=1)
data = data.drop('상태', axis=1)
data = data.drop('위치' , axis=1)
print(data)

       파일명   선종    상태                               위치  state  vessel_type  \
0    1.jpg  범장망    표류    LAT N33.22.872 LON E124.5.074      3            4   
1    2.jpg  범장망    이동   LAT N32.48.241 LON E124.38.553      2            4   
2    3.jpg  범장망    조업   LAT N32.55.893 LON E124.33.731      1            4   
3    4.jpg   타망    이동   LAT N35.40.891 LON E124.25.481      2            1   
4    5.jpg   타망    이동   LAT N32.23.668 LON E125.33.053      2            1   
5    6.jpg   타망    이동   LAT N32.59.352 LON E124.29.609      2            1   
6    7.jpg   유망    조업   LAT N34.54.232 LON E124.42.394      1            2   
7    8.jpg   유망    조업   LAT N34.41.829 LON E124.20.120      1            2   
8    9.jpg   유망    표류   LAT N36.56.279 LON E124.39.427      3            2   
9   10.jpg  안강망    조업   LAT N34.15.543 LON E125.41.175      1            5   
10  11.jpg   통발  \t이동   LAT N34.17.855 LON E124.15.509      2            7   
11  12.jpg   통발    이동   LAT N33.41.223 LON E124.12.877      2   

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 사전 학습된 ResNet50 모델 로드 (ImageNet 데이터셋으로 학습됨)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 마지막 레이어를 교체해 새로운 모델 구성
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링
x = Dense(1024, activation='relu')(x)  # 완전 연결 레이어
predictions = Dense(2, activation='softmax')(x)  # 클래스 개수에 맞춘 출력 레이어

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 사전 학습된 레이어는 학습되지 않도록 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터셋 전처리 (ImageDataGenerator를 사용하여 이미지 증식)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    '샘플데이터셋', target_size=(224, 224), batch_size=32, class_mode='categorical'
)

# 모델 학습
model.fit(train_generator, steps_per_epoch=100, epochs=10)

# 학습된 모델 저장  
model.save('resnet50_finetuned.h5')


Found 12 images belonging to 2 classes.
Epoch 1/10


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 592us/step - accuracy: 0.5000 - loss: 0.6834
Epoch 2/10


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 2.3141  
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 0.7300  
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 1.5630  
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 74us/step - accuracy: 0.5000 - loss: 1.7348  
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 71us/step - accuracy: 0.5000 - loss: 1.1250  
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 81us/step - accuracy: 0.7500 - loss: 0.6331  
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 1.0517  
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 81us/step - accuracy: 0.5000 - loss: 1.2312  
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 71us/step - accuracy: 0.5000 - loss: 1.0289  


In [12]:
import folium

# 좌표 데이터 (위도, 경도)
locations = [
    {"name": "차", "lat": 32 + 11/60, "lon": 125 + 25/60},
    {"name": "카", "lat": 33 + 20/60, "lon": 124 + 8/60},
    {"name": "타", "lat": 34 + 0/60, "lon": 124 + 0/60 + 30/3600},
    {"name": "파", "lat": 35 + 0/60, "lon": 124 + 7/60 + 30/3600},
    {"name": "하", "lat": 35 + 30/60, "lon": 124 + 30/60},
    {"name": "거", "lat": 36 + 45/60, "lon": 124 + 30/60},
    {"name": "너", "lat": 37 + 0/60, "lon": 124 + 20/60},
    {"name": "바", "lat": 32 + 11/60, "lon": 126 + 45/60},
]

# 초기 지도의 중심 좌표 (평균 값 사용)
center_lat = sum([loc["lat"] for loc in locations]) / len(locations)
center_lon = sum([loc["lon"] for loc in locations]) / len(locations)

# 지도 생성
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# 각 위치에 마커 추가
for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=loc["name"]
    ).add_to(mymap)

# 지도 저장
mymap.save("map.html")

# 지도를 출력하고 싶다면, 주피터 노트북 환경에서는 다음 코드로 출력 가능
mymap


In [14]:
pip install pytesseract opencv-python pandas

Note: you may need to restart the kernel to use updated packages.Collecting pytesseract




[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
